# Lab Assignment 6
#### This Lab task is to completed individually, Please Add your Name and ID Below, please comment your code thoroughly and name your variables wisely. (Grade out of 23, 3 marks for commented organized code.)
- Name: Fatimah Almusawi
- ID: 9000400

### Question 1- DATA INTEGRATION:  (5)
Check the files inside microbiome folder, (MID1, MID2, ... , MID9.xls files), you need to combine those files into single dataframe.<br>
Then You will need to add to this dataframe the relevant metadata information from metadata.csv <br>
Each folder represents information from different bacteria, MID1, MID2, ...
<br>
__Inspect the files before answering the question__


## Step_1: Read the data:

In [34]:
import pandas as pd
import os

# lets give a names to the columns
column_names = ['name', 'count']

folder_path = 'csvs\microbiome'  #   folder location
xls_files = [f for f in os.listdir(folder_path) if f.endswith('.xls')]

dataframes = {}  # Dictionary to store each DataFrame

for file in xls_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_excel(file_path, header=None, names=column_names)
    dataframes[file] = df

# Example: Access the DataFrame for 'MID1.xls'
mid1_df = dataframes['MID1.xls']
print(mid1_df.head())  # Display the first few rows




#read data from metadata file
metadata = pd.read_csv('csvs/metadata.csv')
metadata  # Display the first few rows


                                                name  count
0  Archaea "Crenarchaeota" Thermoprotei Desulfuro...      7
1  Archaea "Crenarchaeota" Thermoprotei Desulfuro...      2
2  Archaea "Crenarchaeota" Thermoprotei Sulfoloba...      3
3  Archaea "Crenarchaeota" Thermoprotei Thermopro...      3
4  Archaea "Euryarchaeota" "Methanomicrobia" Meth...      7


,BARCODE,GROUP,SAMPLE
0,MID1,EXTRACTION CONTROL,NaN
1,MID2,NEC 1,tissue
2,MID3,Control 1,tissue
3,MID4,NEC 2,tissue
4,MID5,Control 2,tissue
5,MID6,NEC 1,stool
6,MID7,Control 1,stool
7,MID8,NEC 2,stool
8,MID9,Control 2,stool


now we read all the xls files inside the microbiome file.as i could undrestand from the data, each of this file (MID1,MID2,MID3,..) represent data which coresponde to spisific group.there are 2 major group :NEC and control. and its apear that the 
NEC (Necrotizing Enterocolitis) are the group that represents samples collected from subjects diagnosed with NEC, which is a serious gastrointestinal problem mostly seen in premature infants and the Control are from subjects without NEC, serving as a baseline to compare against the NEC samples.
 for expamle the file MID2 has data thats diagnosed with NEC and was colected from tissue (as sample), MID3 diagnosed as normal or unaffected by the condition being studied and was also colcted from tissue.
 

## step_2: Combining 

Now, let's combine all these MID files together using pandas.concat, which is use for combining DataFrames either by rows or columns. I created a new column to the combined DataFrame. Initially, I simply merged all the .xls files together. However, when I attempted to merge this combined DataFrame with another DataFrame containing metadata, I encountered difficulties. To resolve this, I decided to add a new column that specifies the barcode each row originates from. This addition makes it easier to merge with the metadata DataFrame.

In [36]:
for barcode, df in dataframes.items():
    # Add a 'BARCODE' column with the file name (barcode) as its value
    df['BARCODE'] = barcode

combined_df = pd.concat(dataframes.values(),ignore_index=True,axis=0)

combined_df['BARCODE'] = combined_df['BARCODE'].str.replace('.xls', '', regex=False)

combined_df

,name,count,BARCODE
0,"Archaea ""Crenarchaeota"" Thermoprotei Desulfuro...",7,MID1
1,"Archaea ""Crenarchaeota"" Thermoprotei Desulfuro...",2,MID1
2,"Archaea ""Crenarchaeota"" Thermoprotei Sulfoloba...",3,MID1
3,"Archaea ""Crenarchaeota"" Thermoprotei Thermopro...",3,MID1
4,"Archaea ""Euryarchaeota"" ""Methanomicrobia"" Meth...",7,MID1
...,...,...,...
2391,"Bacteria ""Thermotogae"" Thermotogae Thermotogal...",43,MID9
2392,"Bacteria ""Thermotogae"" Thermotogae Thermotogal...",16,MID9
2393,"Bacteria ""Verrucomicrobia"" Opitutae Opitutales...",1,MID9
2394,Bacteria Cyanobacteria Cyanobacteria Chloropl...,1,MID9


## step_3: Merging:
now lets merge the combined one with metadata datafram,using pand.merge.<br>
The `how='left'` parameter in the `pd.merge()` function specifies that the merge operation should include all rows from the left DataFrame (`combined_df`), appending information from the right DataFrame (`metadata`) where there is a match on the specified key column(s). The `on='BARCODE'` parameter indicates that the merge should be performed based on the 'BARCODE' column present in both DataFrames, using its values to identify which rows correspond to each other across the two datasets. This approach ensures that each sample's data in `combined_df` is augmented with the relevant metadata from `metadata`, based on the unique identifier provided by the 'BARCODE', while preserving all original samples regardless of whether matching metadata exists.


In [38]:
#merge combined_df and metadata
merged_df =pd.merge(combined_df, metadata,how='left', on='BARCODE')
merged_df

,name,count,BARCODE,GROUP,SAMPLE
0,"Archaea ""Crenarchaeota"" Thermoprotei Desulfuro...",7,MID1,EXTRACTION CONTROL,NaN
1,"Archaea ""Crenarchaeota"" Thermoprotei Desulfuro...",2,MID1,EXTRACTION CONTROL,NaN
2,"Archaea ""Crenarchaeota"" Thermoprotei Sulfoloba...",3,MID1,EXTRACTION CONTROL,NaN
3,"Archaea ""Crenarchaeota"" Thermoprotei Thermopro...",3,MID1,EXTRACTION CONTROL,NaN
4,"Archaea ""Euryarchaeota"" ""Methanomicrobia"" Meth...",7,MID1,EXTRACTION CONTROL,NaN
...,...,...,...,...,...
2391,"Bacteria ""Thermotogae"" Thermotogae Thermotogal...",43,MID9,Control 2,stool
2392,"Bacteria ""Thermotogae"" Thermotogae Thermotogal...",16,MID9,Control 2,stool
2393,"Bacteria ""Verrucomicrobia"" Opitutae Opitutales...",1,MID9,Control 2,stool
2394,Bacteria Cyanobacteria Cyanobacteria Chloropl...,1,MID9,Control 2,stool


### Question 2- Feature Engineering - Attribute Correlation (15)
- Load the file income_data.csv, inspect the columns.
- Determine if race and education features have a statistical relationship. 
- Explain and conclude your findings (7 till here)
- Inspect the dataset carefully and create an Indicator Feature (0/1) --(3 marks here)
- Inspect the dataset carefully and create an Interaction Feature (Additional feature engineered from your dataset)-- (3 marks here)<br>
__The features you create must be resonable, please add an explanation to explain why did you chose those features__  (2)

In [44]:
#read income_data.csv
income_data = pd.read_csv('csvs/income_data.csv')
income_data.head() # Display the first few rows




,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [45]:
print(income_data.columns)

Index(['age', 'workclass', 'fnlwgt', 'education', 'educational-num',
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')


##  statistical relationship:
To determine if there's a statistical relationship between race and education, we can use the Chi-Square test of independence.

In [46]:
from scipy.stats import chi2_contingency

# Create a contingency table
contingency_table = pd.crosstab(income_data['race'], income_data['education'])

# Perform the Chi-Square Test
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi2 Statistic: {chi2}, p-value: {p}")


Chi2 Statistic: 1040.6622432490267, p-value: 7.472985630264855e-179


The Chi2 statistic is significantly large, and the p-value is extremely small (far less than 0.05).Given these results, we can conclude that there is a statistically significant relationship between race and education in our dataset. This means that the distribution of education levels across different races is not due to random chance; instead, there is a clear association between these two variables.




## Indicator Features:
aftrer Inspecting the dataset i decided to make "income" as the Indicator Feature. because it already devided the data to: income>50000 and income<50000. so we dont need to ajust threshold from out own, it's already has.

In [53]:
# Create a new column 'income_above_50K' where 1 indicates income > $50,000, and 0 indicates otherwise
income_data['income_above_50K'] = (income_data['income'] == '>50K').astype(int)
income_data.head()  # Display the first few rows

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income,income_above_50K
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K,0
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K,0
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K,1
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K,1
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K,0


## Interaction Feature:
An interaction feature combines two or more features, potentially uncovering effects that the individual features might not show on their own.<br>
in our case there are many option for Interaction feature. however it is better to go with numberic feature. i will go with educational-num and hours-per-week.

In [60]:

income_data['edu_hours_interaction'] = income_data['educational-num'] * income_data['hours-per-week']
income_data
# Inspect the new feature
print(income_data[['educational-num', 'hours-per-week', 'edu_hours_interaction']].head())

   educational-num  hours-per-week  edu_hours_interaction
0                7              40                    280
1                9              50                    450
2               12              40                    480
3               10              40                    400
4               10              30                    300


the out put shows the successful creation and inspection of the edu_hours_interaction feature within my income_data DataFrame. This new feature is the result of multiplying educational-num by hours-per-week for each individual in the dataset. 

here i will make another interaction feature which is : age and hours per week. i find this new feature usfull as This combination could provide insights into how work hours and age together impact other variables, such as income, health, work-life balance, or career stage.

In [61]:
income_data['age_hours_interaction'] = income_data['age'] * income_data['hours-per-week']

# Inspect the new feature
print(income_data[['age', 'hours-per-week', 'age_hours_interaction']].head())

   age  hours-per-week  age_hours_interaction
0   25              40                   1000
1   38              50                   1900
2   28              40                   1120
3   44              40                   1760
4   18              30                    540


The age_hours_interaction shows how much age and work hours together add up for each person. Bigger numbers mean either older age, more work hours, or both. This helps us see how age and work time might affect things like money earned or job experience